In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import set_config
set_config(transform_output='pandas')
from sklearn.impute import SimpleImputer

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate,StratifiedKFold, cross_val_score, train_test_split
import os
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder,MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor

In [2]:
os.chdir('C:/Users/Katharina/Desktop/Weiterbildung/Bootcamp/Bootcamp/Final_project/data')
data = pd.read_csv('ML_Data_Teams.csv', sep=';')	

In [3]:
data1 = data[['@PositionInEntry',
       '@Rank', '@EarnedPointsTeam', '@EarningsTotalTeam', 'Gender_x', 'Type',
       'SpikeFault', 'SpikePoint', 'ServeFault', 'ServePoint',
       'ServeTotal', 'BlockPoint', 'BlockTotal', 'DigTotal', 'ReceptionFault',
       'SpikeTotal', '@LocalDate', '@LocalTime', 'FederationCode_y',
       'FirstName', 'LastName', 'FirstName2', 'LastName2',   '@DurationSet1',
       '@DurationSet2', '@DurationSet3', 'temperature_2m', 'precipitation',
       'wind_speed_10m', 'rain', 'wind_gusts_10m', 'TeamFault_team',
       'match_win',  'Standard_Namen', 'Standard_Namen_team2']]
#'TournamentNo',  '@PointsTeamASet1',
       # '@PointsTeamBSet1', '@PointsTeamASet2', '@PointsTeamBSet2',
       # '@PointsTeamASet3', '@PointsTeamBSet3',

In [4]:
#Variablen umbenennen
data1 = data1.rename(columns={'Standard_Namen': 'Team1', 'Standard_Namen_team2': 'Team2'})

In [11]:
#Daten für app

special_feature = '@DurationSet3'
data2 = data1.copy()
# Erzeuge den Indikator: 1, wenn in @DurationSet3 ein Wert vorhanden ist, sonst 0.
data2[special_feature + '_indicator'] = data2[special_feature].notnull().astype(int)

#Daten für app

data2.to_csv('ML_SpikeFault_MitWetter.csv', index=False, sep=';')

In [6]:
y = data2.pop('SpikeFault')
X = data2.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

In [7]:
X_train.columns

Index(['@PositionInEntry', '@Rank', '@EarnedPointsTeam', '@EarningsTotalTeam',
       'Gender_x', 'Type', 'SpikePoint', 'ServeFault', 'ServePoint',
       'ServeTotal', 'BlockPoint', 'BlockTotal', 'DigTotal', 'ReceptionFault',
       'SpikeTotal', '@LocalDate', '@LocalTime', 'FederationCode_y',
       'FirstName', 'LastName', 'FirstName2', 'LastName2', '@DurationSet1',
       '@DurationSet2', '@DurationSet3', 'temperature_2m', 'precipitation',
       'wind_speed_10m', 'rain', 'wind_gusts_10m', 'TeamFault_team',
       'match_win', 'Team1', 'Team2', '@DurationSet3_indicator'],
      dtype='object')

In [8]:
# import numpy as np
# import pandas as pd
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.impute import KNNImputer, SimpleImputer
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# Bestimme nun die nominalen und numerischen Spalten anhand von X_train (nachdem die Indikatorspalte hinzugefügt wurde)
nominal_features = X_train.select_dtypes(include=['object']).columns.tolist()
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Erstelle eine Liste der allgemeinen numerischen Features, exklusive des speziellen Features und dessen Indikator
general_numeric_features = [col for col in numeric_features if col not in [special_feature, special_feature + '_indicator']]

# Erstelle separate Pipelines:
# 1. Für allgemeine numerische Features mit KNNImputer
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', StandardScaler())
])

# 2. Für das spezielle Feature, das immer mit 0 imputiert werden soll (das signalisiert, dass kein dritter Satz stattgefunden hat)
special_numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', StandardScaler())
])

# 3. Für die Indikatorspalte (@DurationSet3_indicator)
indicator_transformer = Pipeline(steps=[
    # Hier ist meist keine Imputation nötig; Skalierung ist optional, da 0 und 1 oft schon aussagekräftig sind.
    ('scaler', StandardScaler())
])

# 4. Für nominale Features: Imputation mit dem häufigsten Wert + OneHotEncoding
nominal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Baue den ColumnTransformer unter Berücksichtigung aller Feature-Gruppen:
transformers = [
    ('num', numeric_transformer, general_numeric_features),
    ('spec_num', special_numeric_transformer, [special_feature]),
    ('indicator', indicator_transformer, [special_feature + '_indicator']),
    ('nom', nominal_transformer, nominal_features)
]

preprocessor = ColumnTransformer(transformers=transformers)

# # Beispiel: Angenommen, X ist dein Input-DataFrame

# # Benutzerdefinierter Transformer, der eine Indikatorspalte hinzufügt
# class AddMissingIndicator(BaseEstimator, TransformerMixin):
#     def __init__(self, column):
#         self.column = column  # z. B. '@DurationSet3'
        
#     def fit(self, X, y=None):
#         return self
    
#     def transform(self, X):
#         X = X.copy()
#         # Indikatorspalte: 1, falls ein Wert vorhanden, 0 falls NaN
#         indicator_name = self.column + '_indicator'
#         X[indicator_name] = np.where(X[self.column].isna(), 0, 1)
#         return X

# # Definiere den speziellen numerischen Feature-Namen, der immer mit 0 imputiert werden soll.
# special_feature = '@DurationSet3'

# # Wende den Transformer für die Indikatorspalte an.
# # Dieser Schritt fügt automatisch die neue Spalte '@DurationSet3_indicator' zu X hinzu.
# indicator_pipeline = Pipeline(steps=[
#     ('add_indicator', AddMissingIndicator(special_feature))
# ])

# # Wende diesen Schritt direkt vor dem eigentlichen Preprocessing an:
# X_transformed = indicator_pipeline.fit_transform(X)

# # Jetzt ermittelst du die Listen der Spalten.
# nominal_features = X_transformed.select_dtypes(include=['object']).columns.tolist()
# numeric_features = X_transformed.select_dtypes(include=['int64', 'float64']).columns.tolist()

# # Da du für das spezielle Feature gesondert verfahren möchtest, trenne es ab:
# general_numeric_features = [col for col in numeric_features if col not in [special_feature, special_feature + '_indicator']]

# # Pipeline für general numerische Features (mit KNNImputer)
# numeric_transformer = Pipeline(steps=[
#     ('imputer', KNNImputer(n_neighbors=5)),
#     ('scaler', StandardScaler())
# ])

# # Pipeline für das spezielle Feature, das mit 0 imputiert wird
# special_numeric_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
#     ('scaler', StandardScaler())
# ])

# # Falls du auch die Indikatorspalte skalieren möchtest (in vielen Fällen kann man sie aber auch unskaliert lassen):
# indicator_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='constant', fill_value=0)),  # hier eigentlich nicht nötig
#     ('scaler', StandardScaler())
# ])

# # Pipeline für nominale Features
# nominal_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
# ])

# # Baue den ColumnTransformer, der jetzt vier Gruppen kombiniert:
# # 1. Allgemeine numerische Features  
# # 2. Das spezielle Feature (@DurationSet3)
# # 3. Die Indikatorspalte (@DurationSet3_indicator)
# # 4. Nominale Features
# transformers = [
#     ('num', numeric_transformer, general_numeric_features),
#     ('spec_num', special_numeric_transformer, [special_feature]),
#     ('indicator', indicator_transformer, [special_feature + '_indicator']),
#     ('nom', nominal_transformer, nominal_features)
# ]

# preprocessor = ColumnTransformer(transformers=transformers)

# # # Nun kannst du den vollständigen Preprocessing-Schritt anwenden:
# # X_preprocessed = preprocessor.fit_transform(X_transformed)
# # print("Shape der transformierten Daten:", X_preprocessed.shape)


In [ ]:
# # select categorical and numerical column names
# nominal_features = X.select_dtypes(include=['object']).columns.tolist()
# # Define feature types
# numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# # Create preprocessors for different feature types
# numeric_transformer = Pipeline(steps=[
#     ('imputer', KNNImputer(n_neighbors=5)),
#     ('scaler', StandardScaler())
# ])

# nominal_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
# ])

# # Build the column transformer
# transformers = [
#     ('num', numeric_transformer, numeric_features),
#     ('nom', nominal_transformer, nominal_features)
# ]

# preprocessor = ColumnTransformer(transformers=transformers)

In [9]:
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error

model_scores = []
def score_model(model_name, y_true, y_pred):
  scores = {
      'Model': model_name,
      'MAE ($)': round(mean_absolute_error(y_true, y_pred), 2),
      'RMSE ($)': round(root_mean_squared_error(y_true, y_pred), 2),
      'MAPE (%)': round(100 * mean_absolute_percentage_error(y_true, y_pred), 2),
      'R-Squared': round(r2_score(y_true, y_pred), 3)
  }
  return scores

In [12]:
from sklearn.ensemble import GradientBoostingRegressor
gbr_pipeline = make_pipeline(preprocessor,
                             RobustScaler(),
                             GradientBoostingRegressor(random_state=42, n_estimators=200, learning_rate=0.1, 
                                                  max_depth=5, subsample=0.8))

GB = gbr_pipeline.fit(X_train, y_train)

gbr_predictions = gbr_pipeline.predict(X_test)

model_scores.append(score_model('GradientBoostigRegressor', y_test, gbr_predictions))
pd.DataFrame(model_scores)

,Model,MAE ($),RMSE ($),MAPE (%),R-Squared
0,GradientBoostigRegressor,1.85,2.36,1.743184e+16,0.605


In [13]:
import joblib
# Modell speichern als .pkl-Datei
joblib.dump(GB, "GradientBoosting_SpikeFault_mitWetter_NEUEDATEN.pkl")

['GradientBoosting_SpikeFault_mitWetter_NEUEDATEN.pkl']

In [14]:

# 1. Extrahiere das trainierte Model (der letzte Schritt in der Pipeline)
model = gbr_pipeline.named_steps['gradientboostingregressor']

# 2. Extrahiere den Preprocessor-Schritt, der die Features formatiert hat.
# Je nachdem, wie deine Pipeline aufgebaut ist, kann der Schritt-Name variieren.
# Beim make_pipeline werden die Namen automatisch generiert:
preprocessor = gbr_pipeline.named_steps[list(gbr_pipeline.named_steps.keys())[1]]
# Alternativ, wenn du die Pipeline explizit mit Namen erstellt hast, z.B. im Pipeline-Konstruktur,
# dann wäre es etwas wie:
preprocessor = gbr_pipeline.named_steps['columntransformer']

# 3. Hole die Feature-Namen aus dem Preprocessor:
# Diese Methode funktioniert, wenn der Preprocessor und seine Zwischenschritte get_feature_names_out unterstützen.
feature_names = preprocessor.get_feature_names_out()

# 4. Extrahiere die Feature Importances aus dem Model
importances = model.feature_importances_

# 5. Erstelle einen DataFrame zum besseren Überblick
feature_importances_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print(feature_importances_df.head(10))


                    feature  importance
14          num__SpikeTotal    0.196617
23           num__match_win    0.167709
22      num__TeamFault_team    0.121967
24  spec_num__@DurationSet3    0.115445
6           num__SpikePoint    0.046143
4             num__Gender_x    0.038915
17      num__temperature_2m    0.026728
12            num__DigTotal    0.021103
9           num__ServeTotal    0.019774
19      num__wind_speed_10m    0.014516


In [15]:
train_r2 = gbr_pipeline.score(X_train, y_train)
test_r2  = gbr_pipeline.score(X_test, y_test)
print("Train R²:", train_r2)
print("Test R²:", test_r2)

Train R²: 0.7625067245968157
Test R²: 0.6047769284022164


In [16]:
from sklearn.ensemble import RandomForestRegressor
rf_pipeline = make_pipeline(preprocessor,
                             RobustScaler(),
                             RandomForestRegressor(random_state=42, n_estimators=200, max_depth=15, 
                                         min_samples_split=5, min_samples_leaf=2))

RF = rf_pipeline.fit(X_train, y_train)

rf_predictions = rf_pipeline.predict(X_test)

model_scores.append(score_model('RandomForest', y_test, rf_predictions))
pd.DataFrame(model_scores)

,Model,MAE ($),RMSE ($),MAPE (%),R-Squared
0,GradientBoostigRegressor,1.85,2.36,1.743184e+16,0.605
1,RandomForest,1.91,2.47,1.531171e+16,0.568


In [17]:
import joblib
# Modell speichern als .pkl-Datei
joblib.dump(RF, "RandomForest_SpikeFault_mitWetter_NEUEDATEN.pkl")

['RandomForest_SpikeFault_mitWetter_NEUEDATEN.pkl']

In [18]:
train_r2 = rf_pipeline.score(X_train, y_train)
test_r2  = rf_pipeline.score(X_test, y_test)
print("Train R²:", train_r2)
print("Test R²:", test_r2)

Train R²: 0.8533211303997992
Test R²: 0.5684126744330557


In [19]:
# 1. Extrahiere das trainierte Model (der letzte Schritt in der Pipeline)
model = rf_pipeline.named_steps['randomforestregressor']

# 2. Extrahiere den Preprocessor-Schritt, der die Features formatiert hat.
# Je nachdem, wie deine Pipeline aufgebaut ist, kann der Schritt-Name variieren.
# Beim make_pipeline werden die Namen automatisch generiert:
preprocessor = rf_pipeline.named_steps[list(rf_pipeline.named_steps.keys())[1]]
# Alternativ, wenn du die Pipeline explizit mit Namen erstellt hast, z.B. im Pipeline-Konstruktur,
# dann wäre es etwas wie:

preprocessor = rf_pipeline.named_steps['columntransformer']

# 3. Hole die Feature-Namen aus dem Preprocessor:
# Diese Methode funktioniert, wenn der Preprocessor und seine Zwischenschritte get_feature_names_out unterstützen.
feature_names = preprocessor.get_feature_names_out()

# 4. Extrahiere die Feature Importances aus dem Model
importances = model.feature_importances_

# 5. Erstelle einen DataFrame zum besseren Überblick
feature_importances_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print(feature_importances_df.head(10))


                    feature  importance
23           num__match_win    0.168638
14          num__SpikeTotal    0.156417
24  spec_num__@DurationSet3    0.134319
22      num__TeamFault_team    0.104731
6           num__SpikePoint    0.046508
17      num__temperature_2m    0.037529
4             num__Gender_x    0.033372
16       num__@DurationSet2    0.023098
19      num__wind_speed_10m    0.021967
15       num__@DurationSet1    0.020915
